In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df_unique = df.nunique()
df_unique

In [ ]:
#Binary variables
binary_variable = list(df_unique[df_unique==2].index)
binary_variable

In [ ]:
categorical_variables = list(df_unique[(df_unique>2)&(df_unique <=6)].index)
df[categorical_variables].nunique()

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, LabelBinarizer
lb, le = LabelBinarizer(), LabelEncoder()

#transforming ordinal/categorical variables with multiple categoris 

for columns in categorical_variables:
    df[columns] = le.fit_transform(df[columns])

In [ ]:
df[categorical_variables].astype('category').describe()

In [ ]:
#encoding binary variables
for columns in binary_variable:
    df[columns] = lb.fit_transform(df[columns])

In [ ]:
df[binary_variable].astype('category').describe()

In [ ]:
df['TotalCharges'] = pd.to_numeric(df.TotalCharges,errors='coerce')

In [ ]:
df.dtypes

In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df['TotalCharges'] = df['TotalCharges'].fillna(df.TotalCharges.median())

In [ ]:
df.isnull().sum()

In [ ]:
df.corrwith(df.Churn).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(df.corr(),annot=True)
plt.ylim(20,0)

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot( x='Contract',
    y='MonthlyCharges',
    hue='Churn',
    data=df );

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_validate

X = df.iloc[:, 1:-1]
y = df.iloc[:,-1]

In [ ]:
df['Churn'].value_counts(normalize=True)

In [ ]:
from imblearn import over_sampling,under_sampling
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_validate

scaler= MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True, stratify=y)
X_train =scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  **1) With Grandient Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'learning_rate': [ 0.5, 0.2, 0.1, 0.01, 0.001], 
              'subsample':[1.0, 0.5,0.2], 
              'max_features':[4, 5, 10, 12, 19],
              'n_estimators':[15, 20, 30, 40, 80, 100, 200, 400]
             }

GB = GridSearchCV(GradientBoostingClassifier(random_state=142), param_grid=parameters, scoring='accuracy', n_jobs=-1)
GB = GB.fit(X_train, y_train)
y_predGB = GB.predict(X_test)

GB.best_params_

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,y_predGB))

# 2) With Adaboost

In [ ]:
parameters = {'learning_rate': [ 0.5, 0.2, 0.1, 0.01, 0.001], 
              #'subsample':[1.0, 0.5,0.2], 
              #'max_features':[4, 5, 10, 12, 19],
              'n_estimators':[15, 20, 30, 40, 80, 100, 200, 400]
             }

AB = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, scoring='accuracy', n_jobs=-1)
AB = AB.fit(X_train,y_train)
y_predAB = AB.predict(X_test)

GB.best_params_

In [ ]:
print(classification_report(y_test,y_predAB))

# 3) With Logistic Regression penalty L2

In [ ]:
from sklearn.linear_model import LogisticRegression

L2 = LogisticRegression(penalty='l2',solver='saga', max_iter=700)
L2 = L2.fit(X_train,y_train)

y_predl2 = L2.predict(X_test)

print(classification_report(y_test,y_predl2))

# 4) With Decision Tree and Feature Importance

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier().fit(X_train,y_train)

params_grid = {'max_depth':range(1, DT.tree_.max_depth+1, 2),
               'max_features':range(1,len(DT.feature_importances_)+1)
              }

DT_GV = GridSearchCV(DecisionTreeClassifier(random_state=101),
                   param_grid=params_grid,
                   scoring='accuracy',
                   n_jobs=-1)

DT_GV = DT_GV.fit(X_train,y_train)
y_predDT = DT_GV.predict(X_test)
DT_GV.best_estimator_

In [ ]:
Feature_importance = pd.Series(DT.feature_importances_, index=[x for x in X]).sort_values(ascending=False)
bar = Feature_importance.plot(kind='bar', figsize=(18,10))
bar.set(xlabel='Features')
bar.set(ylabel='Relative_Importance')

In [ ]:
print(Feature_importance)

# 5) Stacking all these up with Voting classifier for final prediction - Gradient Boosting + Adaboost + L2 Reg + DT

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = [('GB', GB),('AB', AB), ('L2', L2), ('DT_GV', DT_GV) ]

VC = VotingClassifier(classifiers,voting='soft')

VC = VC.fit(X_train,y_train)
y_predvc = VC.predict(X_test)

print(classification_report(y_test,y_predvc))

In [ ]:
cm = confusion_matrix(y_test,y_predvc)
plt.figure(figsize=(16,6))
sns.set_context("talk")
#sns.heatmap(cm/np.sum(cm),annot=True, cmap='coolwarm',fmt='.2%')
sns.heatmap(cm,annot=True,fmt='d')
plt.ylim(2,0)

# Thank you!! My first contribution to Kaggle